In [1]:
import sys
sys.path.append('../')
import pandas as pd
import requests
from bs4 import BeautifulSoup
import tqdm.notebook as tq
from modules import get_movie_names

  0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
get_movie_names.fetch_movie_names()
get_movie_names.cleanMovieName()

Fetching 1000 movie names from 10 pages...


100%|██████████| 10/10 [02:03<00:00, 13.39s/it]

1000 movie names fetched and stored as csv under ../data/raw/unclean_movie_names.csv
Cleaning movie names...
Movie names cleaned and stored under ../data/processed/clean_movie_names.csv


In [3]:
!python3 ../modules/prepare_training_data.py

42303it [00:01, 25170.18it/s]
100%|█████████████████████████████████| 42303/42303 [00:00<00:00, 880877.55it/s]


In [4]:
BASE_DIR='../data/processed/'
movies_list= BASE_DIR + 'clean_movie_names.csv'
BASE_URL='https://en.wikipedia.org/wiki/'
DEST_DIR = '../data/unseen/'

def fetch_attributes_from_wikipedia():
    cast=[]
    release_year=[]
    plot=[]
    title=[]
    movies_name=pd.read_csv(movies_list)
    print("Fetching movie attributes..")
    for movie in tq.tqdm(movies_name['text']):
        full_plot="NA"
        movie_uri=movie.replace(" ","_")
        start_url = requests.get(BASE_URL + movie_uri).content
        soup = BeautifulSoup(start_url, 'html.parser')
        table = soup.find("table",{"class":"infobox vevent"})
        if table is None:
            start_url = requests.get(BASE_URL + movie + '_(film)').content
            soup = BeautifulSoup(start_url, 'html.parser')
            table = soup.find("table",{"class":"infobox vevent"})
        values = []
        keys=[]
        if table is not None:
            try:
                tag  = soup.select_one('#Plot').find_parent('h2').find_next_sibling()
                while tag.name == 'p':
                    full_plot=full_plot + tag.text
                    tag = tag.find_next_sibling()
            except:
                full_plot=full_plot
            for tr in table:
                th=tr.find_all('th')
                td = tr.find_all('td')
                value = [tr.text for tr in td]
                key= [tr.text for tr in th]
                values.append(value)
                keys.append(key)
            values=values[0]
            keys=keys[0]
            dictionary = dict(zip(keys, values))
            try:
                cast.append(dictionary['Starring'])
                release_year.append(dictionary['Release date'])
                title.append(movie)
                plot.append(full_plot)
                #print("All attributes fetched for movie - "+movie)
            except:
                cast.append('NA')
                release_year.append('NA')
                title.append(movie)
                plot.append(full_plot)
    print(len(title))
    print(len(cast))
    print(len(release_year))
    print(len(plot))
    unseen_data = pd.DataFrame({'title':title,
                                'cast':cast,'release_year':release_year,'plot':plot})
    unseen_data.to_csv(DEST_DIR+'unseen_data.csv',index=False)
    print("Data fetching from Wikipedia completed! Results stored under ../data/unseen/")

In [ ]:
fetch_attributes_from_wikipedia()

Fetching movie attributes..
